In [84]:
api_key="ImPu9SCg8pZaGJyaQZ6X6LK3FP7O6HdSMpQdG3eax6nGIJEwPA4ryyFRN1MSfYg7"
api_secret="xHKeKf7Zz0FGTrhuL8aP7yvLo8e9jm8MlLi6JuWYgFdMyX1q3evdzEbPrt1ZCRvY"

In [85]:
from binance.client import Client
client = Client(api_key, api_secret)
import time
import dateparser
import pytz
from datetime import datetime
import json

In [16]:
str(datetime.now())

'2020-04-26 01:02:47.021848'

## Save Prices

In [2]:
#!/home/ubuntu/anaconda3/bin/python

api_key="ImPu9SCg8pZaGJyaQZ6X6LK3FP7O6HdSMpQdG3eax6nGIJEwPA4ryyFRN1MSfYg7"
api_secret="xHKeKf7Zz0FGTrhuL8aP7yvLo8e9jm8MlLi6JuWYgFdMyX1q3evdzEbPrt1ZCRvY"
from binance.client import Client
client = Client(api_key, api_secret)
import time
import dateparser
import pytz
from datetime import datetime
import json
import sys
import pandas as pd

outdir = "/home/ubuntu/binance_data/"
outfile = outdir + "binance_minute_prices_2020.txt"
def save_prices(outfile):
    prices = pd.DataFrame.from_dict(client.get_all_tickers())
    prices['price_time'] = str(datetime.now())
    prices.to_csv(outfile, mode='a', header=False, index=False, sep="\t")

save_prices(outfile)

outfile = outdir + "binance_minute_orders_2020.txt"

def save_orders(outfile):
    odf=pd.DataFrame.from_dict(client.get_orderbook_tickers())
    odf['order_time'] = str(datetime.now())
    odf.to_csv(outfile, mode='a', header=False, index=False, sep="\t")
save_orders(outfile)

outfile = outdir + "binance_futures_minute_orders_2020.txt"

def save_futures_orders(outfile):
    fob = pd.DataFrame.from_dict(client.futures_orderbook_ticker())
    fob['order_time'] = str(datetime.now())
    fob.to_csv(outfile, mode='a', header=False, index=False, sep="\t")
save_futures_orders(outfile)



outfile = outdir + "binance_futures_minute_prices_2020.txt"

def save_futures_prices(outfile):
    fprice = pd.DataFrame.from_dict(client.futures_symbol_ticker())
    fprice['order_time'] = str(datetime.now())
    fprice.to_csv(outfile, mode='a', header=False, index=False, sep="\t")
save_futures_prices(outfile)


ModuleNotFoundError: No module named 'binance.client'; 'binance' is not a package

#### Add the following to a cron job using: `crontab -e`

```
* * * * * /home/ubuntu/anaconda3/bin/python /home/ubuntu/python/binance_save_all.py

```

## Save Klines

In [1]:
#!/home/ubuntu/anaconda3/bin/python
from binance.client import Client
import time
import dateparser
import pytz
from datetime import datetime
import json

def date_to_milliseconds(date_str):
    """Convert UTC date to milliseconds
    If using offset strings add "UTC" to date string e.g. "now UTC", "11 hours ago UTC"
    See dateparse docs for formats http://dateparser.readthedocs.io/en/latest/
    :param date_str: date in readable format, i.e. "January 01, 2018", "11 hours ago UTC", "now UTC"
    :type date_str: str
    """
    # get epoch value in UTC
    epoch = datetime.utcfromtimestamp(0).replace(tzinfo=pytz.utc)
    # parse our date string
    d = dateparser.parse(date_str)
    # if the date is not timezone aware apply UTC timezone
    if d.tzinfo is None or d.tzinfo.utcoffset(d) is None:
        d = d.replace(tzinfo=pytz.utc)

    # return the difference in time
    return int((d - epoch).total_seconds() * 1000.0)


def interval_to_milliseconds(interval):
    """Convert a Binance interval string to milliseconds
    :param interval: Binance interval string 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w
    :type interval: str
    :return:
         None if unit not one of m, h, d or w
         None if string not in correct format
         int value of interval in milliseconds
    """
    ms = None
    seconds_per_unit = {
        "m": 60,
        "h": 60 * 60,
        "d": 24 * 60 * 60,
        "w": 7 * 24 * 60 * 60
    }

    unit = interval[-1]
    if unit in seconds_per_unit:
        try:
            ms = int(interval[:-1]) * seconds_per_unit[unit] * 1000
        except ValueError:
            pass
    return ms

def get_historical_klines(symbol, interval, start_str, end_str=None):
    """Get Historical Klines from Binance
    See dateparse docs for valid start and end string formats http://dateparser.readthedocs.io/en/latest/
    If using offset strings for dates add "UTC" to date string e.g. "now UTC", "11 hours ago UTC"
    :param symbol: Name of symbol pair e.g BNBBTC
    :type symbol: str
    :param interval: Biannce Kline interval
    :type interval: str
    :param start_str: Start date string in UTC format
    :type start_str: str
    :param end_str: optional - end date string in UTC format
    :type end_str: str
    :return: list of OHLCV values
    """
    # create the Binance client, no need for api key
    client = Client("", "")

    # init our list
    output_data = []

    # setup the max limit
    limit = 500

    # convert interval to useful value in seconds
    timeframe = interval_to_milliseconds(interval)

    # convert our date strings to milliseconds
    start_ts = date_to_milliseconds(start_str)

    # if an end time was passed convert it
    end_ts = None
    if end_str:
        end_ts = date_to_milliseconds(end_str)

    idx = 0
    # it can be difficult to know when a symbol was listed on Binance so allow start time to be before list date
    symbol_existed = False
    while True:
        # fetch the klines from start_ts up to max 500 entries or the end_ts if set
        temp_data = client.get_klines(
            symbol=symbol,
            interval=interval,
            limit=limit,
            startTime=start_ts,
            endTime=end_ts
        )

        # handle the case where our start date is before the symbol pair listed on Binance
        if not symbol_existed and len(temp_data):
            symbol_existed = True

        if symbol_existed:
            # append this loops data to our output data
            output_data += temp_data

            # update our start timestamp using the last value in the array and add the interval timeframe
            start_ts = temp_data[len(temp_data) - 1][0] + timeframe
        else:
            # it wasn't listed yet, increment our start date
            start_ts += timeframe

        idx += 1
        # check if we received less than the required limit and exit the loop
        if len(temp_data) < limit:
            # exit the while loop
            break

        # sleep after every 3rd call to be kind to the API
        if idx % 3 == 0:
            time.sleep(1)

    return output_data


symfile = pd.read_csv("/home/ubuntu/binance_data/binance_tickers.txt", header=None, sep="\t", names=['symbol'])
syms = [i for i in set(symfile.symbol.values)]
outfile = "/home/ubuntu/binance_data/klines_usdsymbols.txt"

for symbol in syms:
    klinesa = get_historical_klines(symbol, Client.KLINE_INTERVAL_1MINUTE, "5 minutes ago UTC")
    dfout = pd.DataFrame.from_dict(klinesa)
    dfout['symbol'] = symbol
    dfout['time'] = str(datetime.now())
    dfout.to_csv(outfile, mode='a', sep="\t", header=False, index=False)

ModuleNotFoundError: No module named 'binance'

In [34]:
symbol

NameError: name 'symbol' is not defined

In [35]:
symbol = "ETHBTC"
klinesa = get_historical_klines(symbol, Client.KLINE_INTERVAL_1MINUTE, "5 minutes ago UTC")
dfout = pd.DataFrame.from_dict(klinesa)
dfout['symbol'] = symbol
dfout['time'] = str(datetime.now())
dfout.head()
# dfout.to_csv(outfile, mode='a', sep="\t", header=False, index=False)

,0,1,2,3,4,5,6,7,8,9,10,11,symbol,time
0,1587876240000,0.02572900,0.02573200,0.02572300,0.02572300,23.24000000,1587876299999,0.59783266,16,4.83800000,0.12447778,0,ETHBTC,2020-04-26 04:48:50.727164
1,1587876300000,0.02572200,0.02572800,0.02572200,0.02572400,51.92000000,1587876359999,1.33566795,13,26.91600000,0.69249483,0,ETHBTC,2020-04-26 04:48:50.727164
2,1587876360000,0.02572400,0.02573100,0.02571600,0.02571900,79.39900000,1587876419999,2.04215849,39,16.61600000,0.42747934,0,ETHBTC,2020-04-26 04:48:50.727164
3,1587876420000,0.02572600,0.02572600,0.02570700,0.02570900,198.67600000,1587876479999,5.10832701,58,75.02900000,1.92931731,0,ETHBTC,2020-04-26 04:48:50.727164
4,1587876480000,0.02571400,0.02571800,0.02570900,0.02571600,49.94100000,1587876539999,1.28405295,30,17.92600000,0.46095680,0,ETHBTC,2020-04-26 04:48:50.727164


In [31]:
dfout.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,1587789360000,0.02498100,0.02499300,0.02498100,0.02498100,12.17400000,1587789419999,0.30417229,16,5.74500000,0.14356650,0
1,1587789420000,0.02498500,0.02499100,0.02498100,0.02498100,14.02400000,1587789479999,0.35045160,12,13.66700000,0.34153323,0
2,1587789480000,0.02498100,0.02500100,0.02497600,0.02498600,18.18700000,1587789539999,0.45460897,19,15.00400000,0.37507888,0
3,1587789540000,0.02500300,0.02500400,0.02498600,0.02499000,31.33300000,1587789599999,0.78313180,17,5.73500000,0.14337721,0
4,1587789600000,0.02500200,0.02501500,0.02499200,0.02500000,24.82000000,1587789659999,0.62077476,20,21.00200000,0.52532541,0


In [3]:
klinesa = get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC")
dfout = pd.DataFrame.from_dict(klinesa)
dfout.head()




In [29]:
symdf = pd.read_csv("/home/ubuntu/binance_data/binance_minute_prices_2020.txt", sep="\t", header=None, names=['price', 'symbol', 'time'])
symbols = [i for i in set(symdf.symbol.values)if "USD" in i or "BTC" in i]
symout = pd.DataFrame(pd.Series(symbols))
symout.to_csv("/home/ubuntu/binance_data/binance_tickers.txt", sep="\t", header=False, index=False)

In [30]:
symfile = pd.read_csv("/home/ubuntu/binance_data/binance_tickers.txt", header=None, sep="\t", names=['symbol'])
syms = [i for i in set(symfile.symbol.values)]
print(len(syms))
syms[:5]

465


['RENUSDT', 'ENJBTC', 'LUNBTC', 'BTSBTC', 'STXUSDT']

In [28]:
symusd = [i for i in syms if "USD" in i or "BTC" in i]
print(len(symusd))

465
